<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C10-Windows_Privilege_Escalation/Creating_A_Process_Monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Creating a Process Monitor__*

Several years ago, Justin, one of the authors of this book, contributed to El Jefe, a project of the security provider Immunity. At its core, El Jefe is a very simple process-monitoring system. The tool is designed to help people on defensive teams track process creation and the installation of malware. While consulting one day, his coworker Mark Wuergler suggested that they use El Jefe offensively: With it, they could monitor processes executed as SYSTEM on the target Windows machines. This would provide insight into potentially insecure file handling or child process creation. It worked, and they walked away with numerous privilege escalation bugs, giving them the keys to the kingdom.

The major drawback of the original El Jefe was that it used a DLL, injected into every process, to intercept calls to the native __CreateProcess__ function. It then used a named pipe to communicate with the collection client, which forwarded the details of the process creation to the logging server. Unfortunately, most antivirus software also hooks the __Create_Process__ calls, so either they view you as malware or you have system instability issues when running El Jefe side by side with the antivirus software.

We'll re-create some of El Jefe's monitoring capabilities in a hookless manner, gearing it toward offensive techniques. This should make our monitoring portable and give us the ability to run it alongside antivirus software without issue.

### *__Process Monitoring with WMI__*

The Windows Management Instrumentation (__WMI__) API gives programmers the ability to monitor a system for certain events and then receive callbacks when those events occur. We'll leverage this interface to receive a callback every time a process is created and then log some valuable information: The time the process was created, the user who spawned the process, the executable that was launched and its command line arguments, the process ID, and the parent process ID. This will show us any processes created by higher privilege accounts, and in particular, any processes that were created as a regular user but have been granted additional Windows privileges that you can leverage.

Let's begin by writing a very simple monitoring script that provides the basic process information and then build on that to determine the enabled privileges. This code was adapted from the Python WMI page (http://timgolden.me.uk/python/wmi/tutorial.html). Note that in order to capture information about high-privilege processes created by SYSTEM, for example, you'll need to run your monitoring script as Administrator. Start by adding the following code to __process_monitor.py__.

In [ ]:
import os
import sys
import win32api
import win32con
import win32security
import wmi

def log_to_file(message):
    with open('process_monitor_log.csv', 'a') as fd:
        fd.write(f'{message}\r\n')

def monitor():
    head = 'CommandLine, Time, Executable, Parent PID, PID, User, Privileges'
    log_to_file(head)
    c = wmi.WMI() #[1]
    process_watcher = c.Win32_Process.watch_for('creation') #[2]
    while Ture:
        try:
            new_process = process_watcher() #[3]
            cmdline = new_process.CommandLine
            create_date = new_process.CreationDate
            executable = new_process.ExecutablePath
            parent_pid = new_process.ParentProcessId
            pid = new_process.ProcessId
            proc_owner = new_process.GetOwner() #[4]

            privileges = 'N/A'
            process_log_message = (
                f'{cmdline} , {create_date} , {executable},'
                f'{parent_pid} , {pid} , {proc_owner} , {privileges}'
            )
            print(process_log_message)
            print()
            log_to_file(process_log_message)
        except Exception:
            pass

if __name__ == '__main__':
    monitor()

We start by instantiating the WMI class __[1]__ and tell it to watch for the process creation event __[2]__. We then enter a loop, which blocks until __process_watcher__ returns a new process event __[3]__. The new process event is a WMI class called __Win32_Process__ that contains all of the relevant information we're after (see MSDN documentation online for more information on the __Win32_Process__ WMI class). One of the class functions is __GetOwner__, which we call __[4]__ to determine who spawned the process. We collect all of the process information we're looking for, output it to the screen, and log it to a file.

### *__Kicking the Tires__*

Let's fire up the process-monitoring script and create some processes to see what the output looks like:

```
C:\Users\tim\work> python process_monitor.py
"Calculator.exe",
20200624083538.964492-240 ,
C:\Program Files\WindowsApps\Microsoft.WindowsCalculator\Calculator.exe,
1204 ,
10312 ,
('DESKTOP-CC91N7I', 0, 'tim') ,
N/A

notepad ,
20200624083340.325593-240 ,
C:\Windows\system32\notepad.exe,
13184 ,
12788 ,
('DESKTOP-CC91N7I', 0, 'tim') ,
N/A
```

After running the script, we ran __notepad.exe__ and __calc.exe__. As you can see, the tool outputs this process information correctly. You could now take an extended break, let this script run for a day, and capture records of all the running processes, scheduled tasks, and various software updaters. You might spot malware if you're (un)lucky. It's also useful to log in and out of the system, as events generated from these actions could indicate privileged processes. Now that we have basic process monitoring in place, let's fill out the privileges field in our logging. First, though, you should learn a little bit about how Windows privileges work and why they're important.